In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from gsm import GaussianSoftmaxModel
from rsm import RecurrentStickbreakingModel
from evaluation import validate, print_flat_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
if config.model == 'gsm':
    Model = GaussianSoftmaxModel
elif config.model == 'rsm':
    Model = RecurrentStickbreakingModel
model = Model(config)    
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topics_freq_tokens = [[idx_to_word[idx] for idx in topic_freq_idxs] for topic_freq_idxs in topics_freq_idxs]
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_flat_topic_sample(sess, model, topics_freq_tokens=topics_freq_tokens)

            # update tree
            if not config.static:
                config.n_topic, update_flg, diff = model.update_topic(sess, dev_batches)
                print('Diff: %.6f' % diff)
                if update_flg:
                    print('Update to %i' % config.n_topic)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = Model(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

display(log_df)
print_flat_topic_sample(sess, model, topics_freq_tokens=topics_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     41   10    9  111.76  595  110.95  0.80  0.01  104.37  506  103.54   
10000    40   20   19  111.46  623  110.49  0.96  0.01  104.11  499  103.07   
15000    43   30   29  111.29  632  110.21  1.07  0.01  104.29  504  103.12   
20000    40   40   39  111.18  634  110.02  1.15  0.00  104.10  500  102.85   
25000    40   50   49  111.09  632  109.88  1.21  0.00  104.04  497  102.76   
30000    37   60   59  111.03  629  109.77  1.25  0.00  104.04  499  102.66   
35000    36   70   69  110.97  625  109.68  1.29  0.00  104.15  503  102.72   
40000    36   80   79  110.93  621  109.59  1.33  0.00  103.99  496  102.54   
45000    36   90   89  110.89  617  109.52  1.36  0.00  103.82  489  102.35   
50000    36  100   99  110.85  613  109.46  1.39  0.00  103.88  490  102.37   
55000    37  110  109  110.82  610  109.41  1.41  0.00  103.78  491  102.27   
60000    37  120  119  110.80  606  109.37  1.43  0.00  103.75  486  102.22   
65000    37  130  129  110.78  604  109.33  1.44  0.00  103.63  484  102.10   
70000    36  140  139  110.76  601  109.29  1.46  0.00  103.70  487  102.17   
75000    36  150  149  110.74  598  109.26  1.47  0.00  103.68  484  102.16   
80000    36  160  159  110.72  596  109.24  1.49  0.00  103.68  487  102.15   
85000    37  170  169  110.71  593  109.21  1.50  0.00  103.69  487  102.16   
90000    37  180  179  110.70  591  109.19  1.51  0.00  103.82  490  102.28   
95000    36  190  189  110.69  590  109.17  1.52  0.00  104.00  500  102.45   
100000   36  200  199  110.68  588  109.15  1.52  0.00  103.73  488  102.18   
105000   36  210  209  110.67  586  109.13  1.53  0.00  103.65  485  102.09   
110000   36  220  219  110.66  584  109.12  1.54  0.00  103.83  492  102.26   
115000   37  230  229  110.65  583  109.10  1.55  0.00  103.65  486  102.08   
120000   36  240  239  110.64  581  109.09  1.55  0.00  103.79  487  102.20   
125000   37  250  249  110.63  580  109.07  1.56  0.00  103.76  489  102.19   
130000   36  260  259  110.63  579  109.06  1.57  0.00  103.75  488  102.17   
135000   37  270  269  110.62  577  109.05  1.57  0.00  103.80  491  102.20   
140000   36  280  279  110.61  576  109.04  1.58  0.00  103.45  478  101.86   
145000   36  290  289  110.61  575  109.02  1.58  0.00  103.73  487  102.13   
150000   37  300  299  110.60  573  109.01  1.59  0.00  103.60  483  102.00   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   37  701  200  110.48  549  108.78  1.70  0.00  103.53  481  101.82   
355000   37  711  210  110.48  549  108.78  1.70  0.00  103.55  482  101.84   
360000   36  721  220  110.48  548  108.77  1.70  0.00  103.67  485  101.96   
365000   44  731  230  110.48  548  108.77  1.70  0.00  103.74  488  102.03   
370000   46  741  240  110.48  548  108.77  1.71  0.00  103.60  484  101.88   
375000   46  751  250  110.47  547  108.76  1.71  0.00  103.53  482  101.81   
380000   46  761  260  110.47  547  108.76  1.71  0.00  103.67  488  101.95   
385000   46  771  270  110.47  547  108.76  1.71  0.00  103.51  478  101.78   
390000   46  781  280  110.47  547  108.75  1.71  0.00  103.47  482  101.74   
395000   46  791  290  110.47  546  108.75  1.71  0.00  103.61  483  101.87   
400000   46  801  300  110.46  546  108.75  1.72  0.00  103.51  479  101.77   
405000   46  811  310  110.46  546  108.74  1.72  0.00  103.55  481  101.82   
410000   41  821  320  110.46  546  108.74  1.72  0.00  103.66  485  101.92   
415000   37  831  330  110.46  545  108.74  1.72  0.00  103.55  481  101.82   
420000   37  841  340  110.46  545  108.73  1.72  0.00  103.43  479  101.69   
425000   37  851  350  110.46  545  108.73  1.72  0.00  103.47  481  101.73   
430000   37  861  360  110.45  545  108.73  1.73  0.00  103.61  485  101.88   
435000   37  871  370  110.45  544  108.73  1.73  0.00  103.64  485  101.

0 carry pockets room pocket small strap ! nice perfect hold
1 ! cover color love keyboard mac pro perfectly hard recommend
2 & ; sleeve protection inside air inch pro nice protect
3 bottom months top 've year broke back bought weeks years
4 $ smell amazon item ... return received ordered days ?
5 zipper flap zippers velcro zip close closed open - design
6 reviews read review reading decided based thought mentioned positive found
7 quality close bad thing pretty price ... bought ca stars
8 speck cheap buy plastic rubber cases pretty thin disappointed purchased
9 return poor broke quality returned refund warranty contacted targus disappointed
10 quality made problem 'm time day purchased back high seams
Diff: -0.000006


TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     41   10    9  111.76  595  110.95  0.80  0.01  104.37  506  103.54   
10000    40   20   19  111.46  623  110.49  0.96  0.01  104.11  499  103.07   
15000    43   30   29  111.29  632  110.21  1.07  0.01  104.29  504  103.12   
20000    40   40   39  111.18  634  110.02  1.15  0.00  104.10  500  102.85   
25000    40   50   49  111.09  632  109.88  1.21  0.00  104.04  497  102.76   
30000    37   60   59  111.03  629  109.77  1.25  0.00  104.04  499  102.66   
35000    36   70   69  110.97  625  109.68  1.29  0.00  104.15  503  102.72   
40000    36   80   79  110.93  621  109.59  1.33  0.00  103.99  496  102.54   
45000    36   90   89  110.89  617  109.52  1.36  0.00  103.82  489  102.35   
50000    36  100   99  110.85  613  109.46  1.39  0.00  103.88  490  102.37   
55000    37  110  109  110.82  610  109.41  1.41  0.00  103.78  491  102.27   
60000    37  120  119  110.80  606  109.37  1.43  0.00  103.75  486  102.22   
65000    37  130  129  110.78  604  109.33  1.44  0.00  103.63  484  102.10   
70000    36  140  139  110.76  601  109.29  1.46  0.00  103.70  487  102.17   
75000    36  150  149  110.74  598  109.26  1.47  0.00  103.68  484  102.16   
80000    36  160  159  110.72  596  109.24  1.49  0.00  103.68  487  102.15   
85000    37  170  169  110.71  593  109.21  1.50  0.00  103.69  487  102.16   
90000    37  180  179  110.70  591  109.19  1.51  0.00  103.82  490  102.28   
95000    36  190  189  110.69  590  109.17  1.52  0.00  104.00  500  102.45   
100000   36  200  199  110.68  588  109.15  1.52  0.00  103.73  488  102.18   
105000   36  210  209  110.67  586  109.13  1.53  0.00  103.65  485  102.09   
110000   36  220  219  110.66  584  109.12  1.54  0.00  103.83  492  102.26   
115000   37  230  229  110.65  583  109.10  1.55  0.00  103.65  486  102.08   
120000   36  240  239  110.64  581  109.09  1.55  0.00  103.79  487  102.20   
125000   37  250  249  110.63  580  109.07  1.56  0.00  103.76  489  102.19   
130000   36  260  259  110.63  579  109.06  1.57  0.00  103.75  488  102.17   
135000   37  270  269  110.62  577  109.05  1.57  0.00  103.80  491  102.20   
140000   36  280  279  110.61  576  109.04  1.58  0.00  103.45  478  101.86   
145000   36  290  289  110.61  575  109.02  1.58  0.00  103.73  487  102.13   
150000   37  300  299  110.60  573  109.01  1.59  0.00  103.60  483  102.00   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   37  701  200  110.48  549  108.78  1.70  0.00  103.53  481  101.82   
355000   37  711  210  110.48  549  108.78  1.70  0.00  103.55  482  101.84   
360000   36  721  220  110.48  548  108.77  1.70  0.00  103.67  485  101.96   
365000   44  731  230  110.48  548  108.77  1.70  0.00  103.74  488  102.03   
370000   46  741  240  110.48  548  108.77  1.71  0.00  103.60  484  101.88   
375000   46  751  250  110.47  547  108.76  1.71  0.00  103.53  482  101.81   
380000   46  761  260  110.47  547  108.76  1.71  0.00  103.67  488  101.95   
385000   46  771  270  110.47  547  108.76  1.71  0.00  103.51  478  101.78   
390000   46  781  280  110.47  547  108.75  1.71  0.00  103.47  482  101.74   
395000   46  791  290  110.47  546  108.75  1.71  0.00  103.61  483  101.87   
400000   46  801  300  110.46  546  108.75  1.72  0.00  103.51  479  101.77   
405000   46  811  310  110.46  546  108.74  1.72  0.00  103.55  481  101.82   
410000   41  821  320  110.46  546  108.74  1.72  0.00  103.66  485  101.92   
415000   37  831  330  110.46  545  108.74  1.72  0.00  103.55  481  101.82   
420000   37  841  340  110.46  545  108.73  1.72  0.00  103.43  479  101.69   
425000   37  851  350  110.46  545  108.73  1.72  0.00  103.47  481  101.73   
430000   37  861  360  110.45  545  108.73  1.73  0.00  103.61  485  101.88   
435000   37  871  370  110.45  544  108.73  1.73  0.00  103.64  485  101.

0 carry pockets room pocket small strap ! nice perfect hold
1 ! cover color love keyboard mac pro perfectly hard recommend
2 & ; sleeve protection inside air inch pro nice protect
3 bottom months top 've year broke back bought weeks years
4 $ smell amazon item ... return received ordered days ?
5 zipper flap zippers velcro zip close closed open - design
6 reviews read review reading decided based thought mentioned positive found
7 quality close bad thing pretty price ... bought ca stars
8 speck cheap buy plastic rubber cases pretty thin disappointed purchased
9 return poor broke quality returned refund warranty contacted targus disappointed
10 quality made problem 'm time day purchased back high seams
